In [9]:
low_RAM_mode = True
very_low_RAM = False

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
df = pd.read_csv("avito_train_1kk.tsv",sep='\t')

In [13]:
print (df.shape, df.is_blocked.mean())
df[:5]

(1204949, 13) 0.22822210732570425


,itemid,category,subcategory,title,description,attrs,price,is_proved,is_blocked,phones_cnt,emails_cnt,urls_cnt,close_hours
0,10000010,Транспорт,Автомобили с пробегом,"Toyota Sera, 1991",Новая оригинальная линзованая оптика на ксенон...,"{""Год выпуска"":""1991"", ""Тип кузова"":""Купе"", ""П...",150000,NaN,0,0,0,0,0.03
1,10000094,Личные вещи,"Одежда, обувь, аксессуары",Костюм Steilmann,Юбка и топ из панбархата. Под топ трикотажная...,"{""Вид одежды"":""Женская одежда"", ""Предмет одежд...",1500,NaN,0,0,0,0,0.41
2,10000299,Личные вещи,Детская одежда и обувь,"Костюм Didriksons Boardman, размер 100, краги,...","Костюм Didriksons Boardman, в отличном состоян...","{""Вид одежды"":""Для мальчиков"", ""Предмет одежды...",3000,NaN,0,0,0,0,5.49
3,10000309,Недвижимость,Квартиры,"1-к квартира, 44 м², 9/20 эт.","В кирпичном пан.-м доме, продается одноком.-ая...","{""Тип объявления"":""Продам"", ""Количество комнат...",2642020,NaN,0,1,0,0,22.47
4,10000317,Услуги,Предложения услуг,"Поездки на таможню, печать в паспорте",Поездки на таможню гражданам СНГ для пересечен...,"{""Вид услуги"":""Деловые услуги"", ""Тип услуги"":""...",1500,0.0,1,0,0,0,1.43


In [26]:
no_blocked = df[df.is_blocked == 0].index.tolist()

In [39]:
df_new = df[df.is_blocked == 0].sample(n = len(df[df.is_blocked == 1]))

In [40]:
df_new = df_new.append(df[df.is_blocked == 1])

In [45]:
del df

In [46]:
df = df_new

In [ ]:
df.to_csv("balanced_class.csv", index=False)

In [2]:
df = pd.read_csv("balanced_class.csv")

In [3]:
len(df)

549992

In [4]:
assert df.is_blocked.mean() < 0.51
assert df.is_blocked.mean() > 0.49
assert len(df) <= 560000

print ("All tests passed")

All tests passed


In [5]:
from nltk.tokenize import RegexpTokenizer
from collections import Counter,defaultdict
tokenizer = RegexpTokenizer(r"\w+")

In [6]:
#Dictionary of tokens
token_counts = Counter()

#All texts
all_texts = np.hstack([df.description.values,df.title.values])

#Compute token frequencies
for s in all_texts:
    if type(s) is not str:
        continue
    s = s.lower()
    tokens = tokenizer.tokenize(s)
    for token in tokens:
        token_counts[token] +=1

In [7]:
vocab_size = 200000
tokens = token_counts.most_common(vocab_size)

In [8]:
token_to_id = {t[0]:i+1 for i,t in enumerate(tokens)}
null_token = "NULL"
token_to_id[null_token] = 0

In [9]:
inv_dict = {}
for i, j in token_to_id.items():
    inv_dict[j] = i

In [10]:
def vectorize(strings, token_to_id, max_len=150):
    token_matrix = []
    for s in strings:
        if type(s) is not str:
            token_matrix.append([0]*max_len)
            continue
        s = s.lower()
        tokens = tokenizer.tokenize(s)
        token_ids = list(map(lambda token: token_to_id.get(token,0), tokens))[:max_len]
        token_ids += [0]*(max_len - len(token_ids))
        token_matrix.append(token_ids)

    return np.array(token_matrix)

In [11]:
desc_tokens = vectorize(df.description.values,token_to_id,max_len = 150)
title_tokens = vectorize(df.title.values,token_to_id,max_len = 15)

In [12]:
print ("Размер матрицы:",title_tokens.shape)
for title, tokens in zip(df.title.values[:3],title_tokens[:3]):
    print (title,'->', tokens[:10],'...')

Размер матрицы: (549992, 15)
Электрогитара Fender Squier Telecaster -> [24503 20959 75260 73598     0     0     0     0     0     0] ...
Отдам коляску -> [ 190 2067    0    0    0    0    0    0    0    0] ...
Коттедж 300 м² на участке 8 сот. -> [1115  198   23    4  186   19  151    0    0    0] ...


In [13]:
#All numeric features
df_numerical_features = df[["phones_cnt","emails_cnt","urls_cnt","price"]]

In [14]:
from sklearn.feature_extraction import DictVectorizer

categories = []
data_cat_subcat = df[["category","subcategory"]].values

categories = [{"category":df.category[i], "subcategory":df.subcategory[i]} for i in df.index]

vectorizer = DictVectorizer(sparse=False)
cat_one_hot = vectorizer.fit_transform(categories)
cat_one_hot = pd.DataFrame(cat_one_hot,columns=vectorizer.feature_names_)

In [15]:
df_non_text = pd.merge(
    df_numerical_features,cat_one_hot,on = np.arange(len(cat_one_hot))
)
del df_non_text["key_0"]

In [16]:
#Target variable - whether or not sample contains prohibited material
target = df.is_blocked.values.astype('int32')
#Preprocessed titles
title_tokens = title_tokens.astype('int32')
#Preprocessed tokens
desc_tokens = desc_tokens.astype('int32')

#Non-sequences
df_non_text = df_non_text.astype('float32')

In [17]:
# split train test depending on itemid
itemid = np.array(df.itemid)
quant = int(np.percentile(itemid, 80))

title_ts, title_tr = title_tokens[itemid>quant], title_tokens[itemid<=quant]
desc_ts, desc_tr = desc_tokens[itemid>quant], desc_tokens[itemid<=quant]
target_ts, target_tr = target[itemid>quant], target[itemid<=quant]
non_text_ts, non_text_tr = np.array(df_non_text[itemid>quant]), np.array(df_non_text[itemid<=quant])

In [18]:
#non text features normalization
non_text_ts = non_text_ts / np.linalg.norm(non_text_ts)
non_text_tr = non_text_tr / np.linalg.norm(non_text_tr)

In [19]:
import random
def shuffle(*arg):
    l = len(arg[0])
    a = np.arange(l)
    random.shuffle(a)
    ans = []
    for i in range(len(arg)):
        ans.append(arg[i][a])
    return ans

In [20]:
#shuffle test and train
desc_ts, title_ts, target_ts = shuffle(desc_ts, title_ts, target_ts)
desc_tr, title_tr, target_tr = shuffle(desc_tr, title_tr, target_tr)

## NN architecture

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
from keras.preprocessing.text import text_to_word_sequence
import keras.preprocessing.text
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Input, merge
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from keras.models import Model

Using TensorFlow backend.


In [24]:
from sklearn.metrics import roc_auc_score, accuracy_score
from oracle import APatK, score

In [34]:
input_title = Input(shape=(None,), dtype='int32', name='input_title')
embed_title = Embedding(len(token_to_id)+1, 64, input_length=None, dropout=0.2, name="embed_title")(input_title)
lstm_title = LSTM(64, dropout_W=0.2, dropout_U=0.2, return_sequences=True)(embed_title)
bid_lstm_title = Bidirectional(LSTM(32, dropout=0.2))(lstm_title)

input_desc = Input(shape=(None,), dtype='int32', name='input_desc')
embed_desc = Embedding(len(token_to_id)+1, 64, input_length=None, dropout=0.2, name="embed_desc")(input_desc)
lstm_desc = LSTM(128, dropout_W=0.2, dropout_U=0.2, return_sequences=True, input_shape=(None, 100))(embed_desc)
bid_lstm_desc = Bidirectional(LSTM(64, dropout=0.2))(lstm_desc)


input_non_text = Input(shape=(df_non_text.shape[1], ), name='input_non')
dense_non_text = Dense(4)(input_non_text)

x = merge([bid_lstm_title, bid_lstm_desc, dense_non_text], mode='concat')
x = Dense(32, activation='sigmoid')(x)
x = Dropout(0.2)(x)
out = Dense(1, activation='sigmoid', name='output')(x)

model = Model(input=[input_title, input_desc, input_non_text], output=[out])

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
batch_size = 10000
mini_batch = 100
nb_epoch = 20
for epoch in range(nb_epoch):
    
    #train on random sampled batch
    print("epoch: ", epoch+1)
    index = random.sample([i for i in range(len(title_tr))], batch_size)
    t_tr, d_tr, nt_tr, y = title_tr[index], desc_tr[index], non_text_tr[index], target_tr[index]
    model.fit([t_tr, d_tr, nt_tr], y,
        nb_epoch=1, batch_size=mini_batch)
    
    #evaluation on first eval_s sampled of shuffled test
    eval_s = 5000
    pred = model.predict([title_ts[:eval_s], desc_ts[:eval_s], non_text_ts[:eval_s]], batch_size=32)
    pred_acc = pred > 0.5
    print("auc ", roc_auc_score(target_ts[:eval_s], pred))
    print("acc ", accuracy_score(pred_acc, target_ts[:eval_s]))
    

epoch:  1
Epoch 1/1
10000/10000 [==============================] - 109s - loss: 0.4574 - acc: 0.7631     

In [39]:
#final
pred = model.predict([title_ts, desc_ts, non_text_ts], batch_size=32, verbose=1)
pred_acc = pred > 0.5
print("auc ", roc_auc_score(target_ts, pred))
print("acc ", accuracy_score(pred_acc, target_ts))
print("APatK", APatK(target_ts, pred, K=int(s*0.025)))

109999/109999 [==============================] - 353s     

TypeError: can't multiply sequence by non-int of type 'float'

In [68]:
pred_AP = []
for i in range(len(pred)):
    pred_AP.append(pred[i][0])
pred_AP = np.array(pred_AP) 

In [69]:
print("APatK", APatK(target_ts, pred_AP, K=int(len(pred)*0.025)))

APatK 0.99954928843


In [73]:
print("test_len: ", len(target_ts))

test_len:  109999
